In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import h5py
import mne
import timeit
import random
from scipy.io import savemat
                               

# Preprocessing FST data with mne

In [ ]:
#data will be preprocessed at multiple stages and be saved with the postfix of the corresponding preprocessing step.
root_dir = "file_address";
start = time.time()
subj_names = ['AT','DR','JG','JK']
for subj in subj_names:
    for session in [1,2,3,4]:
        names = glob.glob(root_dir+subj+'/session'+str(session)+'/fst_b*.fif')
        #fname = np.random.choice(names)
        for fname in names:
            raw = mne.io.read_raw_fif(fname, preload=True)
            events = mne.find_events(raw)
            #maxwell sss for finding bad ch & correct using sst
            raw_check = raw.copy().pick_types(exclude=()).filter(None, 40)
            auto_noisy_chs, auto_flat_chs = mne.preprocessing.find_bad_channels_maxwell(raw_check,verbose=False)
            raw.info['bads'].extend(auto_noisy_chs + auto_flat_chs)
            raw_sst = mne.preprocessing.maxwell_filter(raw,st_duration=4,st_correlation=0.95,verbose=False)
            events = mne.find_events(raw_sst)
            picks = mne.pick_types(raw_sst.info, meg=True, eeg=False, eog=False, stim=False)
            epoch = mne.Epochs(raw_sst, events, preload=True, reject=None,picks=picks, baseline=(None,0))
            data=epoch.get_data()
            sio.savemat(fname+'feature_sst.mat',mdict={'data':data})
            print(fname+'feature_sst.mat')
            
            #empty room removal
            system_projs = raw_sst.info['projs']
            raw_sst.del_proj()
            empty_room_file = os.path.join('./'+subj+'/session'+str(session)+'/er.fif')
            empty_room_raw = mne.io.read_raw_fif(empty_room_file)
            empty_room_raw.del_proj()
            empty_room_projs = mne.compute_proj_raw(empty_room_raw)
            n_projs_er = len(empty_room_projs)
            raw_sst.add_proj(empty_room_projs, remove_existing=True)
            events = mne.find_events(raw_sst)
            picks = mne.pick_types(raw_sst.info, meg=True, eeg=False, eog=False, stim=False)
            epoch = mne.Epochs(raw_sst, events, preload=True, reject=None,picks=picks, baseline=(None,0))
            data=epoch.get_data()
            sio.savemat(fname+'feature_sst_empty.mat',mdict={'data':data})
            print(fname+'feature_sst_empty.mat')
            
            #bandwitdh and notch filter
            raw_sst = raw_sst.filter(1,150).notch_filter([60, 120], filter_length='auto', phase='zero')
            events = mne.find_events(raw_sst)
            picks = mne.pick_types(raw_sst.info, meg=True, eeg=False, eog=False, stim=False)
            epoch = mne.Epochs(raw_sst, events, preload=True, reject=None,picks=picks, baseline=(None,0))
            data=epoch.get_data()       
            sio.savemat(fname+'feature_sst_empty_filter.mat',mdict={'data':data})
            print(fname+'feature_sst_empty_filter.mat')
           
            #removing ecg artifacts (heartbeat)
            projs, events = compute_proj_ecg(raw_sst)
            ecg_projs = projs[n_projs_er:]
            n_projs_ecg = len(ecg_projs)
            raw_sst.add_proj(ecg_projs, remove_existing=False)
            raw_sst = raw_sst.filter(1,150).notch_filter([60, 120], filter_length='auto', phase='zero')
            events = mne.find_events(raw_sst)
            picks = mne.pick_types(raw_sst.info, meg=True, eeg=False, eog=False, stim=False)
            epoch = mne.Epochs(raw_sst, events, preload=True, reject=None,picks=picks, baseline=(None,0))
            data=epoch.get_data()          
            sio.savemat(fname+'feature_sst_empty_filter_ecg.mat',mdict={'data':data})
            print(fname+'feature_sst_empty_filter_ecg.mat')
    
            #removing eog artifacts (blink)
            projs, events = compute_proj_eog(raw_sst)
            eog_projs = projs[(n_projs_er+n_projs_ecg):]
            raw_sst.add_proj(eog_projs, remove_existing=False)
            raw_sst = raw_sst.filter(1,150).notch_filter([60, 120], filter_length='auto', phase='zero')
            events = mne.find_events(raw_sst)
            picks = mne.pick_types(raw_sst.info, meg=True, eeg=False, eog=False, stim=False)
            epoch = mne.Epochs(raw_sst, events, preload=True, reject=None,picks=picks, baseline=(None,0))
            data=epoch.get_data()  
            sio.savemat(fname+'feature_sst_empty_filter_ecg_eog.mat',mdict={'data':data})
            print(fname+'feature_sst_empty_filter_ecg_eog.mat')
            
end = time.time()
print(end - start)